In [ ]:
MCS 471 Project One: Laguerre’s Method
The goal of the project is to study the method of Laguerre with Julia programs.
0. Derivation of Laguerre’s Method
Let p be a polynomial in one variable x of degree d. Then its highest degree coefficient cd is nonzero. Assume cd = 1, otherwise, divide every coefficient of p by cd. By the fundamental theorem of algebra, we may then write p as
	p(x) = (x − r1)(x − r2)···(x − rd),	rk ∈ C,	k = 1,2,...,d,	(1)
where r1, r2, ..., rd are the roots of p. Consider the natural logarithm of p:
	L(x) = ln(p(x)) = ln(x − r1) + ln(x − r2) + ··· + ln(x − rd)	(2)
and its derivative
	 .	(3)
Observe that L0(x) = p0(x)/p(x). Consider the second derivative of L, with minus sign, M = −L00,
	 .	(4)
Observe that M(x) = (L0(x))2 − p00(x)/p(x).
Suppose we have a good approximation z0 for the first root r1. Denote δ = z0 − r1 and assume z0 − r2 ≈ ··· ≈ z0 − rd so we may use the same ∆ for all z0 − rk, k 6= 1. With δ and ∆, the
expressions for L0(x) and M(x) respectively in (3) and in (4) at x = z0 simplify to
	 	and  .	(5)
By the two above observations, the values for L0(z0) and M(z0) are computed as p0(z0)/p(z0) and (L0(z0))2 − p00(z0)/p(z0). Knowing δ gives the next approximation for the root r1 as z1 = z0 − δ. We eliminate ∆ using the expression for L0(z0) in (5):
	  ,	(6)
which gives a quadratic equation in the unknown δ. The two solutions are
	 ,	(7)
where the sign ± is chosen to obtain the larger absolute value of the denominator. Equivalently:
	  ,	(8)
where the square root of the complex number is chosen to have a real part that is not negative.
Source: https://en.wikipedia.org/wiki/Laguerre’s method


In [ ]:
Download laguerremethod.jl from the course web site. When executed at the command prompt, or included in a Julia session as below, we see two runs of the method of Laguerre, with 256 bits of precision on Complex{BigFloat} numbers.
julia> include("laguerremethod.jl"); running on x^2 - 3*x + 2 ...
step :	real(root)	imag(root)	|dx|	|p(x)|
0	: 7.6844767519656987e-01 9.4051500071518701e-01
1	: 1.0000000000000000e+00 2.5908505665283334e-77 9.69e-01 3.11e-77succeeded after 1 step(s) running on a random polynomial ...
step :	real(root)	imag(root)	|dx|	|p(x)|
0	: 2.8106589510145752e-01 7.9293102916315772e-01
1	: 3.8195606114724018e-01 1.0558403094632030e+00 2.82e-01 6.12e-01
2	: 3.5653298955097910e-01 1.0201172610276352e+00 4.38e-02 1.27e-03
3	: 3.5662939462136947e-01 1.0201705282073562e+00 1.10e-04 2.23e-11
4	: 3.5662939462020207e-01 1.0201705282088903e+00 1.93e-12 1.19e-34
5	: 3.5662939462020207e-01 1.0201705282088903e+00 1.03e-35 8.64e-77succeeded after 5 step(s)


In [ ]:
. Make a Jupyter notebook with the posted program laguerremethod.jl.
Apply the method of Laguerre to the square root of a number N, running the method on the polynomial x2 − N, starting at z0 = N. Run experiments at a sequence of increasing precision, starting at 256. Do sufficiently many experiments till you can answer the following question.
How many iterations does the method of Laguerre require to compute the square root of a number N with n bits of precision?


In [ ]:
Computing All Roots
If r is a root of p(x), q(x) = p(x)/(x − r), p(x) = q(x)(x − r). Running the method of Laguerre on the quotient q will give the next root of p. Repeating this as long as deg(q) > 2 will give all roots. Assignment Two. Write a function to compute all roots of a polynomial p. For the polynomial division, you could write your own function, but you may also use the Polynomials package.
Demonstrate the correctness by running your function on polynomials with random coefficients. Is the last root computed as accurately as the first root? Verify with the original p.


In [ ]:
The Wilkinson Polynomial
Consider the Wilkinson polynomial wd(x) = (x − 1)(x − 2)···(x − d), for increasing degrees, up to degree 20, and increasing precision. Do sufficiently many experiments to answer the following. Assignment Three. What is the smallest value for the precision to compute roots of wd(x) with at least 8 decimal places correct? Note that for d = 2, this value is smaller than 256 bits. The third assignment can be solved independently from the second assignment.


In [ ]:
EXample
# P-1 MCS 471 due Fri 5 Feb 2021 : laguerremethod.jl

import Random # to fix the seed of the random numbers
using Printf  # for formatted printing of numbers
"""
This simple program illustrates the method of Laguerre
to approximate roots of polynomials in one variable.

To evaluate polynomials, we use evalpoly(x, p),
where x is the value for the input and p[1], p[2], ...
are the coefficients of the polynomial p,
in ascending order of the power of x.  The evalpoly(x, p) 
returns p[1] + p[2]*x + p[3]*x^2 + ... + p[d+1]*x^d,
where d is the degree of p, d = size(p, 1) - 1.

The default type of each coefficient of p is Complex{BigFloat},
complex numbers of arbitrary precision.
The function precision(BigFloat) returns the precision in bits
currently in use.  The default value is 256.
With setprecision(nb), the precision is set to as many bits as
the value of nb.

Three functions are defined: diffpoly, laguerre, and runlaguerre.
"""

"""
The function diffpoly returns the coefficients of the derivative
of the polynomial with coefficients in c. \\
The zero polynomial is represented as an array of one zero.

EXAMPLE : \\
    c = Array([Complex{BigFloat}(i,i) for i = 1:3]) \\
    diffpoly(c)
"""
function diffpoly(c::Array{Complex{BigFloat},1})
    sz = size(c, 1)
    if sz < 2
        result = Array([Complex{BigFloat}(0)])
    else
        result = Array([Complex{BigFloat}(0) for _ = 1:sz-1])
        for i=2:sz
            result[i-1] = (i-1)*c[i]
        end
   end
   return result
end

"""
The function laguerre applies the method of Laguerre.

ON ENTRY : (p, d1p, d2p, z0, dxtol, pxtol, maxit, verbose)         \\
  p        are the coefficients of a polynomial in one variable    \\
  d1p      are the coefficients of the first derivative of p       \\
  d2p      are the coefficients of the second derivative of p      \\
  z0       is an approximation for the root                        \\
  dxtol    is the tolerance on the forward error                   \\
  pxtol    is the tolerance on the backward error                  \\
  maxit    is the maximum number of iterations                     \\
  verbose  is the verbose flag, if true, writes one line each step
                                                                 
ON RETURN : (root, absdx, abspx, nbrit, fail)                      \\
  root     is an approximation for the root                        \\
  absdx    is the estimated forward error                          \\
  abspx    is the estimated backward error                         \\
  nbrit    is the number of iterations                             \\
  fail     is true if tolerances not reached,                      \\
           false otherwise.
"""
function laguerre(p::Array{Complex{BigFloat},1},
                  d1p::Array{Complex{BigFloat},1},
                  d2p::Array{Complex{BigFloat},1},
                  z0::Complex{BigFloat},
                  dxtol::Float64=1.0e-8,
                  pxtol::Float64=1.0e-8,
                  maxit::Int64=10,
                  verbose::Bool=true)
    root = z0; dx = 1; pval = 1
    degm1 = size(p, 1) - 2 # degree of p minus one
    deg = degm1 + 1
    if verbose
        title = "       real(root)             imag(root)"
        println("step : $title          |dx|     |p(x)|")
        stri = @sprintf("%3d", 0)
        strx = @sprintf("%.16e  %.16e", real(root), imag(root))
        println("$stri  : $strx")
    end
    for i=1:maxit
        pval = evalpoly(root, p)
        if abs(pval) < pxtol
            if verbose
                stri = string(i-1)
                println("succeeded after $stri step(s)")
            end
            return (root, abs(dx), abs(pval), i, false)
        end
        d1val = evalpoly(root, d1p)
        d2val = evalpoly(root, d2p)
        Lroot = d1val/pval
        Mroot = Lroot^2 - d2val/pval
        valsqrt = sqrt(degm1*(deg*Mroot - Lroot^2))
        yplus = Lroot + valsqrt
        yminus = Lroot - valsqrt
        if abs(yplus) > abs(yminus)
            dx = deg/yplus
        else
            dx = deg/yminus
        end
        root = root - dx
        pval = evalpoly(root, p)
        if verbose
            stri = @sprintf("%3d", i)
            strx = @sprintf("%.16e  %.16e", real(root), imag(root))
            strdx = @sprintf("%.2e", abs(dx))
            strpx = @sprintf("%.2e", abs(pval))
            println("$stri  : $strx  $strdx  $strpx")
        end
        if abs(dx) < dxtol
            if verbose
                stri = string(i)
                println("succeeded after $stri step(s)")
            end
            return (root, abs(dx), abs(pval), i, false)
        end
    end
    strN = string(maxit)
    println("failed requirements after $strN step(s)")
    return (root, abs(dx), abs(pval), maxit, true)
end

"""
The function runlaguerre demonstrates the method of Laguerre
on the polynomial (x-1)*(x-2) = x^2 - 3*x + 2.
"""
function runlaguerre()
    c1 = Complex{BigFloat}(2)
    c2 = Complex{BigFloat}(-3)
    c3 = Complex{BigFloat}(1)
    p = [c1, c2, c3]
    d1p = diffpoly(p)
    d2p = diffpoly(d1p)
    # we fix the seed for the random number generator
    Random.seed!(123);
    z0 = Complex{BigFloat}(rand(),rand())
    println("running on x^2 - 3*x + 2 ...")
    laguerre(p,d1p,d2p,z0)
    p = [Complex{BigFloat}(rand(), rand()) for _ = 1:10]
    d1p = diffpoly(p)
    d2p = diffpoly(d1p)
    z0 = Complex{BigFloat}(rand(),rand())
    println("running on a random polynomial ...")
    laguerre(p,d1p,d2p,z0,1.0e-60, 1.0e-60)
end

runlaguerre()